In [33]:
!pip install praat-parselmouth
!pip install praat-textgrids

In [19]:
import parselmouth
import textgrids
from IPython.display import Audio

In [ ]:
def create_diphones_dict(diphones) -> dict:
    diphones_dict = {}
    for a in range(len(diphones)-1):
        p1 = diphones[a]
        p2 = diphones[a+1]
        milieu1 = (p1.xmin + p1.xmax)/2
        milieu2 = (p2.xmin + p2.xmax)/2
        diphones_dict[(p1.text, p2.text)] = (milieu1, milieu2)
    return diphones_dict


def get_sampling(milieu1, milieu2, sound):
    # La fréquence d'échantillonage nous permet de rebasculer vers des échantillons, sur lesquels on pourra itérer (il faut que ce soit des nombres entiers)
    start_idx = int(milieu1 * sound.sampling_frequency)
    end_idx = int(milieu2 * sound.sampling_frequency)
    channel = sound.values[0]

    # Si la boucle ne trouve rien, on garde le milieu théorique
    clean_start = milieu1
    clean_end = milieu2

    for b in range(start_idx, end_idx):
        if channel[b] <= 0 and channel[b+1] > 0:
            clean_start = b/sound.sampling_frequency
            break            

    for i in reversed(range(start_idx, end_idx)):
        if channel[i] <= 0 and channel[i+1] > 0:
            clean_end = i/sound.sampling_frequency
            break
    
    return clean_start, clean_end
    

In [16]:
# PPT diapo 10 : d'autres façons de faire une jonction propre (cf fade in/out et overlap)
# Mesurer la similarité entre deux signaux
# Normalisation RMS (root.min.square (en Pascal)) = intensité dans sa valeur réelle (déciBel)
# --> unité perceptive plus facile à manipuler car correspond à ce que l'humain peut percevoir.

segments = [
    (111.84, 112.03),
    (231.85, 232.05),
    (224.37, 224.57),
    (156.37, 156.55),
    ]

extracted = []
fs = sound.sampling_frequency
overlap_ms = 10
N = int(fs * overlap_ms / 1000)
search_ms = 20
M = int(fs * search_ms / 1000)
hann = np.hanning(2 * N)
fade_in  = hann[:N]
fade_out = hann[N:]
for start, end in segments:
    segment = sound.extract_part(from_time=start, to_time=end, preserve_times=False)
    extracted.append(segment.values[0])
    result = extracted[0]
    for seg in extracted[1:]:
        # zones de recouvrement
        tail = result[-N:]
        search_zone = seg[:M]
        # corrélation pour trouver le meilleur point d’alignement
        corr = np.correlate(search_zone, tail, mode="valid")
        best_offset = np.argmax(corr)
        # segment réaligné
        seg_aligned = seg[best_offset:]
        head = seg_aligned[:N]
        crossfade = tail * fade_out + head * fade_in
        result = np.concatenate([
        result[:-N],
        crossfade,
        seg_aligned[N:]
        ])
    new_sound = parselmouth.Sound(
        values=result[np.newaxis, :],
        sampling_frequency=fs
)
ipd.display(Audio(new_sound.values, rate=new_sound.sampling_frequency))


NameError: name 'np' is not defined

In [24]:
def main():
    son = "logatomes.wav"
    grille = "logatomes.TextGrid" # attention aux majuscules
    sound = parselmouth.Sound(son)
    segmentation = textgrids.TextGrid(grille)
    diphones = segmentation["phonemes"]
    chaine_sampa = "sEtynafER"

    #start_time, end_time = 111.84, 112.03

    diphones_dict = create_diphones_dict(diphones)
    liste_morceaux = []

    for i in range(len(chaine_sampa)-1):
        phoneme1 = chaine_sampa[i]
        phoneme2 = chaine_sampa[i+1]

        # On vérifie que le couple recherché se trouve bien dans le dictionnaire
        if (phoneme1, phoneme2) in diphones_dict:
            milieu1, milieu2 = diphones_dict[(phoneme1, phoneme2)]
        
            # Calcul des coupes propres
            clean_start, clean_end = get_sampling(milieu1, milieu2, sound)
    
            # Extraction du morceau
            extraction = sound.extract_part(from_time=clean_start, to_time=clean_end, preserve_times=False)
            
            # Ajout à la liste finale
            liste_morceaux.append(extraction)
        
            print(f"Diphone '{phoneme1}-{phoneme2}' ajouté.")

        else:
            print(f"Diphone '{phoneme1}-{phoneme2}' introuvable")
    
    if liste_morceaux:
        # On colle tous les morceaux ajoutés à la liste
        son_final = parselmouth.Sound.concatenate(liste_morceaux)
        # On sauvegarde le résultat
        son_final.save("resultat_synthese.wav", "WAV")

    display(Audio("resultat_synthese.wav"))

In [25]:
if __name__ == "__main__":
    main()

Diphone 's-E' ajouté.
Diphone 'E-t' ajouté.
Diphone 't-y' ajouté.
Diphone 'y-n' ajouté.
Diphone 'n-a' ajouté.
Diphone 'a-f' ajouté.
Diphone 'f-E' ajouté.
Diphone 'E-R' ajouté.
